In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# date = "lr_1e-06_a_5.0_b_0.5_thsh_5.0"
# folder = os.path.join("result", date)
# print(folder)

In [2]:
def filtering(df):
    # max iat > min iat
    df_filtered = df[df['max iat'] >= df['min iat']]
    # max pkt len > min pkt len
    df_filtered = df_filtered[df_filtered['max pkt_length'] >=  df_filtered['min pkt_length']]
    # pkt count > 0
    df_filtered = df_filtered[df_filtered['packet count'] >= df_filtered['fin count']]
    df_filtered = df_filtered[df_filtered['packet count'] >= df_filtered['syn count']]
    df_filtered = df_filtered[df_filtered['packet count'] >= df_filtered['psh count']]
    df_filtered = df_filtered[df_filtered['packet count'] >= df_filtered['ack count']]
    
    
    # [FIN, ACK]
    df_filtered = df_filtered[df_filtered['fin count'] <= df_filtered['ack count']]
    # [PSH, ACK]
    df_filtered = df_filtered[df_filtered['psh count'] <= df_filtered['ack count']]
    
    # 新增條件 1: 若 pkt count > 0，則 flow duration > 0
    df_filtered = df_filtered[~((df_filtered['packet count'] > 0) & (df_filtered['flow duration'] <= 0))]

    # 新增條件 2: 若 flow duration > 0，則 pkt count > 0
    df_filtered = df_filtered[~((df_filtered['flow duration'] > 0) & (df_filtered['packet count'] <= 0))]

    # 新增條件 3: flow duration >= max iat
    df_filtered = df_filtered[df_filtered['flow duration'] >= df_filtered['max iat']]

    # 新增條件 4: 所有 flags 加總 <= packet count
    df_filtered = df_filtered[
        (df_filtered['fin count'] + df_filtered['syn count'] +
         df_filtered['psh count'] + df_filtered['ack count']) <= (df_filtered['packet count'] * 1.38)
    ]
        
    
#     df_filtered = df_filtered[~((df_filtered['packet count'] > 0) & (df_filtered['fin count'] + df_filtered['syn count'] +
#          df_filtered['psh count'] + df_filtered['ack count'] <= 0))]
#     df_filtered = df_filtered[~((df_filtered['packet count'] <= 0) & (df_filtered['fin count'] + df_filtered['syn count'] +
#          df_filtered['psh count'] + df_filtered['ack count'] > 0))]


#     df_filtered = df_filtered[
#         (df_filtered['fin count'] + df_filtered['syn count'] +
#          df_filtered['psh count'] + df_filtered['ack count']) >= df_filtered['packet count'] * 0.5
#     ]
    return df_filtered

In [3]:
# features=[
#                 'flow duration', 'packet count', 'max pkt_length', 'min pkt_length',
#                 'max iat', 'min iat', 'fin count', 'syn count', 'psh count', 
#                 'ack count', "predict", "loss", "gen_loss", "bd_loss"
#                 ]

features=[
                'flow duration', 'packet count', 'max pkt_length', 'min pkt_length',
                'max iat', 'min iat', 'fin count', 'syn count', 'psh count', 
                'ack count', "predict", "gen_loss",	"disc_loss"
                ]

In [4]:
def main(folder):
    for file in os.listdir(folder):
        if file.startswith("generated_data"):
            file_path = os.path.join(folder, file)
            df = pd.read_csv(file_path, header=None, names=features, skiprows=1)
            print("-" * 10)
            print(f'Filtering {file} ...')
            print(f'number of origin data: {len(df)}')
            df_0 = df[df['predict'] == 0]
            print(f'number of label-0 data: {len(df_0)}')
            
    #         print(df)
            df_filtered = filtering(df_0)
            print(f'number of filered data: {len(df_filtered)}')
            percentage = len(df_filtered) / len(df) * 100
            print(f'Percentage: {percentage:.2f}\%')
            # append origin data
            origin = df.iloc[[0]]
            df_filtered = pd.concat([origin, df_filtered], ignore_index=True)
            output_dir = os.path.join(folder, 'filtered_data')
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, f'filtered_{file}')
            df_filtered.to_csv(output_path, mode='w', header=features, index=False, encoding="utf-8")
    print("Done!")

In [5]:
root = "result"
date = "lr_1e-06_"
for dir in os.listdir(root):
    if dir.startswith(date):
        folder = os.path.join(root, dir)
        print(folder)
        main(folder)

result\lr_1e-06_a_0.3_b_1.0_thsh_5.0
----------
Filtering generated_data_84.csv ...
number of origin data: 5001
number of label-0 data: 91
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_85.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_86.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_87.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_88.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_89.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
Done!
result\lr_1e-06_a_5.0_b_0.5_thsh_5.0
----------
Filtering generated_data_0.csv .

----------
Filtering generated_data_56.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_57.csv ...
number of origin data: 5001
number of label-0 data: 1235
number of filered data: 675
Percentage: 13.50\%
----------
Filtering generated_data_58.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_59.csv ...
number of origin data: 5001
number of label-0 data: 1674
number of filered data: 674
Percentage: 13.48\%
----------
Filtering generated_data_6.csv ...
number of origin data: 5001
number of label-0 data: 0
number of filered data: 0
Percentage: 0.00\%
----------
Filtering generated_data_60.csv ...
number of origin data: 5001
number of label-0 data: 858
number of filered data: 279
Percentage: 5.58\%
----------
Filtering generated_data_61.csv ...
number of origin data: 5001
number of label-0 data: 1272
numbe